In [22]:
pip install lime

  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=e80eb8d0f057bb352956594237aa9ee9239771fd8ed9a80dc0df213cca8085ad
  Stored in directory: /Users/christiantam/Library/Caches/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime
Note: you may need to restart the kernel to use updated packages.


In [23]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, Lasso
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

from lime.lime_text import LimeTextExplainer

import time


In [ ]:
# Binary Classification

num_splits = 5

for i in range(num_splits):
    start_time=time.time()
    train_file_path = f'./F24_Proj3_data/split_{i+1}/train.csv'
    test_file_path = f'./F24_Proj3_data/split_{i+1}/test.csv'
    test_y_file_path = f'./F24_Proj3_data/split_{i+1}/test_y.csv'

    # Load data
    X_train = pd.read_csv(train_file_path).iloc[:, 3:]
    y_train = pd.read_csv(train_file_path).iloc[:, 1]

    X_test = pd.read_csv(test_file_path).iloc[:, 2:]
    y_test = pd.read_csv(test_y_file_path).iloc[:, 1]

    clf = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
    clf.fit(X_train, y_train)

    y_pred_proba = clf.predict_proba(X_test)[:, 1]

    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    print(f'Split {i}: AUC Score for split {i}: {auc_score:.7f} | Execution time : {round(time.time() - start_time, 4)} seconds')


In [ ]:
# Use LogisticRegressionCV with elastic net penalty. Using cross validation and finding the best C and l1 ratio to speed up training.

num_splits = 5
cv = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

for i in range(num_splits):
    start_time = time.time()
    train_file_path = f'./F24_Proj3_data/split_{2}/train.csv'
    test_file_path = f'./F24_Proj3_data/split_{2}/test.csv'
    test_y_file_path = f'./F24_Proj3_data/split_{2}/test_y.csv'

    # Load data
    X_train = pd.read_csv(train_file_path).iloc[:, 3:]
    y_train = pd.read_csv(train_file_path).iloc[:, 1]

    X_test = pd.read_csv(test_file_path).iloc[:, 2:]
    y_test = pd.read_csv(test_y_file_path).iloc[:, 1]

    # LogisticRegressionCV with 'elasticnet' penalty
    log_reg_cv = LogisticRegressionCV(
        penalty='elasticnet',
        solver='saga',
        l1_ratios=[0.1],  
        cv=cv,
        max_iter=1000,
        n_jobs=-1,
        random_state=42
    )
    
    log_reg_cv.fit(X_train, y_train)

    best_C = log_reg_cv.C_
    best_l1_ratio = log_reg_cv.l1_ratio_

    print(f"Split {i+1}: Best C: {best_C} | Best l1_ratio: {best_l1_ratio}")

    y_pred_proba = log_reg_cv.predict_proba(X_test)[:, 1]

    auc_score = roc_auc_score(y_test, y_pred_proba)
    print(f'Split {i+1}: AUC Score for LogisticRegressionCV: {auc_score:.7f} | Execution time: {round(time.time() - start_time, 4)} seconds')

In [2]:
# Use LogisticRegression with best C and l1 ratio we found in the CV approach
num_splits = 5

log_reg = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    l1_ratio=0.1,  
    C=2.7825594,
    max_iter=1000,
    n_jobs=-1,
    random_state=42
)

for i in range(num_splits):
    start_time = time.time()
    train_file_path = f'./F24_Proj3_data/split_{i+1}/train.csv'
    test_file_path = f'./F24_Proj3_data/split_{i+1}/test.csv'
    test_y_file_path = f'./F24_Proj3_data/split_{i+1}/test_y.csv'

    # Load data
    X_train = pd.read_csv(train_file_path).iloc[:, 3:]
    y_train = pd.read_csv(train_file_path).iloc[:, 1]

    X_test = pd.read_csv(test_file_path).iloc[:, 2:]
    y_test = pd.read_csv(test_y_file_path).iloc[:, 1]

    log_reg.fit(X_train, y_train)

    y_pred_proba = log_reg.predict_proba(X_test)[:, 1]

    auc_score = roc_auc_score(y_test, y_pred_proba)
    print(f'Split {i+1}: AUC Score for LogisticRegression: {auc_score:.7f} | Execution time: {round(time.time() - start_time, 4)} seconds')

Split 1: AUC Score for LogisticRegression: 0.9869905 | Execution time: 21.9067 seconds
Split 2: AUC Score for LogisticRegression: 0.9865600 | Execution time: 21.631 seconds
Split 3: AUC Score for LogisticRegression: 0.9862596 | Execution time: 20.7292 seconds
Split 4: AUC Score for LogisticRegression: 0.9867768 | Execution time: 21.2396 seconds
Split 5: AUC Score for LogisticRegression: 0.9862021 | Execution time: 21.731 seconds


In [17]:
train_file_path = f'./F24_Proj3_data/split_1/train.csv'
test_file_path = f'./F24_Proj3_data/split_1/test.csv'
test_y_file_path = f'./F24_Proj3_data/split_1/test_y.csv'

train = pd.read_csv(train_file_path)
test = pd.read_csv(test_file_path)
test_y = pd.read_csv(test_y_file_path)

stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',
             'you', 'your', 'yours', 'their', 'they', 'his', 'her', 'she',
             'he', 'a', 'an', 'and', 'is', 'was', 'are', 'were', 'him',
             'himself', 'has', 'have', 'it', 'its', 'the', 'us']

In [8]:
train['review'] = train['review'].str.replace('&lt;.*?&gt;', ' ', regex=True)
test['review'] = test['review'].str.replace('&lt;.*?&gt;', ' ', regex=True)

vectorizer = CountVectorizer(
    preprocessor=lambda x: x.lower(),  # Convert to lowercase
    stop_words=stop_words,             # Remove stop words
    ngram_range=(1, 4),               # Use 1- to 4-grams
    min_df=0.001,                        # Minimum term frequency
    max_df=0.5,                       # Maximum document frequency
    token_pattern=r"\b[\w+\|']+\b" # Use word tokenizer: See Ethan's comment below
)

dtm_train = vectorizer.fit_transform(train['review'])
# dtm_test = vectorizer.fit_transform(test['review'])

# log_reg.fit(dtm_train, train['sentiment'])

# Make predictions on the test data
# preds = log_reg.predict_proba(dtm_test)[:, 1]  # Get the probabilities for class 1

# print(preds)

In [6]:
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(dtm_train)

lasso_model = Lasso(alpha=0.00623)

lasso_model.fit(X_train_scaled, train['sentiment'])

# Get the coefficients
lasso_coefs = lasso_model.coef_
print('Lasso Coefficients:', lasso_coefs.sum())

# Create a DataFrame with feature names and their coefficients
feature_coef_df = pd.DataFrame(
    {'Feature': np.array(vectorizer.get_feature_names_out()),
     'Coefficient': lasso_coefs})

selected_features = feature_coef_df[feature_coef_df['Coefficient'] != 0]
vocabulary = list(selected_features['Feature'].values)

with open('./myvocab.txt', 'w') as file:
    # Iterate through the list and write each word to a new line
    for word in vocabulary:
        file.write(word + "\n")

len(selected_features)

Lasso Coefficients: -0.2245395020328098


1206

In [35]:
myvocab = open("myvocab.txt", "r").read().splitlines()
tfidf_vectorizer = TfidfVectorizer(vocabulary=myvocab, ngram_range=(1, 4))
dtm_test = tfidf_vectorizer.fit_transform(dtm_train['review'])
# log_reg.fit(dtm_test, test_y['sentiment'])
# print(myvocab)
# print(test_y)
# print(dtm_test)
positive_reviews = test[test_y['sentiment'] == 1].sample(5, random_state=42)
negative_reviews = test[test_y['sentiment'] == 0].sample(5, random_state=42)
selected_reviews = pd.concat([positive_reviews, negative_reviews])

explainer = LimeTextExplainer(class_names=['Negative', 'Positive'])

def explain_review(review_text):
    print(review_text)
    explanation = explainer.explain_instance(review_text, log_reg.predict_proba, num_features=10)
    
    explanation.show_in_notebook(text=True)

for i, row in selected_reviews.iterrows():
    explain_review(row['review'])
    


          id                                             review  embedding_1  \
21221  16917  Well, I fear that my review of this special wo...     0.015161   
1796   42355  What happens when the average joe finds out he...    -0.018924   
3875   10466  For years i've had a distant memory of watchin...     0.021243   
4276   23582  This film is a brilliant retelling of Shakespe...     0.006077   
12025  24055  Mas Oyama was the most successful karate maste...    -0.008232   
23248   7451  I'm a huge fan of both Emily Watson (Breaking ...    -0.015999   
15017  37586  Alan (Anthony Steffen), an English multi-milli...    -0.022035   
23732   5868  This is what Disney Channel shows to kids who ...     0.021406   
15807  48453  You'd think that with Ingrid Bergman and Warne...    -0.011025   
19237  13120  Someone else called this film a \fable-horror\...    -0.014135   

       embedding_2  embedding_3  embedding_4  embedding_5  embedding_6  \
21221     0.012336    -0.050433     0.004414 

TypeError: 'numpy.ndarray' object is not callable